In [1]:
# Step 1: Load Combined QA Pairs
import json
from typing import List

with open("data_preprocessing/output/combined_qa.json", "r", encoding="utf-8") as f:
    qa_data = json.load(f)

In [2]:
# Step 2: Prepare Documents for Embedding
documents = [f"Q: {item['question']}\nA: {item['answer']}" for item in qa_data]

In [11]:
# # Save documents to a JSON file
# with open("data_preprocessing/output/documents.json", "w", encoding="utf-8") as f:
#     json.dump(documents, f, indent=2, ensure_ascii=False)

In [3]:
documents

['Q: What is the profit payment for NUST Asaan Account Savings Account?\nA: It is: Semi-Annually',
 'Q: What is the profit rate for NUST Asaan Account Savings Account?\nA: It is: 19.00%',
 'Q: What is the profit payment for PLS Savings Savings Account?\nA: It is: Semi-Annually',
 'Q: What is the profit rate for PLS Savings Savings Account?\nA: It is: 19.00%',
 'Q: What is the profit payment for Little Champs Account Savings Account?\nA: It is: Semi-Annually',
 'Q: What is the profit rate for Little Champs Account Savings Account?\nA: It is: 19.00%',
 'Q: What is the profit payment for NUST Special Deposit Account (ASDA) Savings Account?\nA: It is: Monthly/Semi-Annually',
 'Q: What is the profit rate for NUST Special Deposit Account (ASDA) Savings Account?\nA: It is: 19.00%',
 'Q: What is the profit payment for NUST Waqaar Account - Senior Citizen Savings Account?\nA: It is: Monthly',
 'Q: What is the profit rate for NUST Waqaar Account - Senior Citizen Savings Account?\nA: It is: 19.00

In [2]:
import requests

In [ ]:
import requests

HUGGINGFACE_API_KEY = API_KEY  # Replace with your Hugging Face API key

def query_qwen_model(prompt: str) -> str:
    API_URL = "https://api-inference.huggingface.co/models/Qwen/Qwen3-4B"
    headers = {
        "Authorization": f"Bearer {HUGGINGFACE_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "inputs": prompt,
        "parameters": {
            "do_sample": True,
            "temperature": 0.7,
            "top_p": 0.95,
            "max_new_tokens": 300
        }
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status()
    result = response.json()

    # Extract response from generated text
    if isinstance(result, list):
        return result[0]["generated_text"].replace(prompt, "").strip()
    elif "error" in result:
        return f"Error: {result['error']}"
    return str(result)


In [4]:
def generate_answer(user_query):
    # (Assuming you already retrieved the context from FAISS or any document store)
    context = retrieve_context(user_query)  # You define this function

    prompt = f"""You are a helpful assistant. Use the following context to answer the question.

    Context:
    {context}

    Question:
    {user_query}

    Answer:"""

    answer = query_qwen_model(prompt)
    return context, answer


In [1]:
# Step 3: Embedding using MiniLM
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(documents, show_progress_bar=True)
embeddings = np.array(embeddings)

c:\Users\jamal\PycharmProjects\rag-llm\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'documents' is not defined

In [5]:
# Step 4: FAISS Index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [6]:
# Save index to a file
faiss.write_index(index, "my_index.faiss")

In [7]:
# Load from the file
index = faiss.read_index("my_index.faiss")

In [8]:
# Step 5: Retrieval Function
def retrieve(query: str, top_k: int = 3) -> List[str]:
    query_vec = embedding_model.encode([query])
    D, I = index.search(query_vec, top_k)
    return [documents[i] for i in I[0]]

In [9]:
# Example Query
query = "What are the features of Little Champs Account?"
top_docs = retrieve(query)

In [10]:
top_docs

['Q: What other Value added features does the Little Champs Account have? for Little Champs Account\nA: Attractive returns on savings account SMS alert service on digital transactions I Net banking services Free education insurance plan – Rs.5,000/- per month for 5 years on savings account & Rs.10,000 per month for 5 years on current account in case of death of the guardian',
 'Q: What are the main Features  of the Little Champs Account. for Little Champs Account\nA: Minimum initial deposit of Rs.100/- Little Champs Savings A/C Free first chequebook* Profit Payment Profit Rate Free debit card* (annual/replacement fees apply). This debit card has the following limits Semi-Annually 0.19 Daily funds Transfer Limit: Rs.100,000/- Daily ATM Withdrawal Limit: Rs.25,000/- Daily POS Limit: Rs.50,000/- * For Current Account only',
 'Q: What is the account type of Little Champs Account is it saving or current ? for Little Champs Account\nA: This account is offered both in current and savings cate

In [ ]:
from huggingface_hub import login
login(HUGGINGFACE_API_KEY)  # Replace with your Hugging Face API key

In [12]:
# Step 6: Load LLM (Llama or Qwen via HuggingFace)
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [21]:
model_name = "deepseek-ai/deepseek-coder-1.3b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [2]:
from huggingface_hub import snapshot_download

local_path = snapshot_download("deepseek-ai/deepseek-coder-1.3b-instruct")
print(local_path)

Fetching 9 files:  11%|█         | 1/9 [00:00<00:05,  1.46it/s]Error while downloading from https://cdn-lfs-us-1.hf.co/repos/ce/fb/cefbbf2dab4f425037697081a326fd0bcaf8af239e502d6573186402ee2a5b31/0f3cee31dbcc5e416a7cc54361675717bf1ac2cce9e651f767b2fc60cba12ff5?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1747752195&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0Nzc1MjE5NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2NlL2ZiL2NlZmJiZjJkYWI0ZjQyNTAzNzY5NzA4MWEzMjZmZDBiY2FmOGFmMjM5ZTUwMmQ2NTczMTg2NDAyZWUyYTViMzEvMGYzY2VlMzFkYmNjNWU0MTZhN2NjNTQzNjE2NzU3MTdiZjFhYzJjY2U5ZTY1MWY3NjdiMmZjNjBjYmExMmZmNT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9uc2UtY29udGVudC10eXBlPSoifV19&Signature=QbwltIRX03glRf%7E003f8%7ENPsD3vKcIT1yDfWqMSyw334oFihPDU5teWiVVtfV4EJNqOadt2a4jxmN-fhnX-5RCd26gDv1EF2sDGwxOQu62UOwnY8

C:\Users\jamal\.cache\huggingface\hub\models--deepseek-ai--deepseek-coder-1.3b-instruct\snapshots\e063262dac8366fc1f28a4da0ff3c50ea66259ca


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_path = r"C:\Users\jamal\.cache\huggingface\hub\models--deepseek-ai--deepseek-coder-1.3b-instruct\snapshots\e063262dac8366fc1f28a4da0ff3c50ea66259ca"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
model_path,
device_map="auto",
torch_dtype=torch.float16
)

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [24]:
# tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-4-mini-reasoning")
# model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-4-mini-reasoning")

# model_id = "meta-llama/Llama-3.1-8B-Instruct"
# model_id = "Qwen/Qwen3-1.7B"
# model_id = "Qwen/Qwen3-4B"

# tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     device_map="auto",
#     torch_dtype=torch.float16,
#     trust_remote_code=True
# )

# model.eval()

In [15]:
from typing import List

def retrieve_and_answer(query: str, top_k: int = 3, max_new_tokens=200) -> str:
    """
    Retrieves top-k most relevant document chunks and generates an answer using an instruction-tuned LLM.
    """
    # Step 1: Encode the query
    query_vec = embedding_model.encode([query])

    # Step 2: Search the FAISS index
    distances, indices = index.search(query_vec, top_k)
    retrieved_chunks = [documents[i] for i in indices[0]]

    # Step 3: Build the prompt
    context = "\n\n".join(retrieved_chunks)
    prompt = f"""You are a helpful bank assistant.
Use the context to answer customer queries clearly and accurately. Also, behave politely. If the question is out-of-domain, say you cannot help with that.

Context:
{context}

Question:
{query}

Answer:"""

    # Step 4: Tokenize and generate response
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)

    # Generate
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=0.7,
        do_sample=True,
        top_k=50,
        top_p=0.95
    )

    # Decode & clean
    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "Answer:" in full_output:
        return full_output.split("Answer:")[-1].strip()
    return full_output.strip()


In [16]:
query = "What is the profit rate for NUST Asaan Account?"
print(retrieve_and_answer(query))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


The profit rate for NUST Asaan Account is 19.00%


In [18]:

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(documents, show_progress_bar=True)
embeddings = np.array(embeddings)

Batches: 100%|██████████| 11/11 [00:13<00:00,  1.20s/it]


In [20]:
embeddings

array([[ 0.01067686,  0.11429954, -0.05263795, ..., -0.00711991,
         0.00535193,  0.00982509],
       [ 0.01119902,  0.09008457, -0.09782137, ..., -0.01622126,
        -0.01364005,  0.02279177],
       [ 0.02479239,  0.11290911, -0.02742065, ...,  0.01231725,
         0.00860872,  0.00312407],
       ...,
       [ 0.00422421, -0.0483009 , -0.02617159, ...,  0.04805793,
         0.10799947,  0.00687814],
       [ 0.00561527,  0.0600477 , -0.05437011, ...,  0.09230928,
         0.06637407,  0.04154782],
       [ 0.03403336, -0.00168171, -0.03410652, ...,  0.05156418,
         0.14887944,  0.03663563]], shape=(335, 384), dtype=float32)

In [19]:
# Step 1: Load and prepare combined_qa.json
def load_qa_pairs(json_path="combined_qa.json"):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    return data  # A list of {"question": ..., "answer": ...}


# Step 2: Embed all QA questions using GEDA-Age (here, SentenceTransformer)
def get_qa_embeddings(qa_pairs):
    questions = [pair["question"] for pair in qa_pairs]
    embeddings = embedding_model.encode(questions, convert_to_tensor=True, normalize_embeddings=True)
    return embeddings


# Step 3: Find Top 3 Most Relevant QA Pairs for a Query
def retrieve_top_k_qa_pairs(query, qa_pairs, k=3):
    qa_embeddings = get_qa_embeddings(qa_pairs)
    query_embedding = embedding_model.encode(query, convert_to_tensor=True, normalize_embeddings=True)

    # Compute cosine similarity
    scores = torch.matmul(qa_embeddings, query_embedding)

    # Get top k indices
    top_k_indices = torch.topk(scores, k=k).indices.tolist()
    
    top_k_pairs = [qa_pairs[i] for i in top_k_indices]
    return top_k_pairs

In [21]:
QA_PATH = "data_preprocessing/output/combined_qa.json"

In [22]:
qa_pairs = load_qa_pairs(QA_PATH)
top_3_context = retrieve_top_k_qa_pairs("What is the current markup rate of ask fauri business finance?", qa_pairs)
top_3_context

[{'question': 'What is the current Mark-up rate of Ask Fauri Business Finance? for NUST Fauri Business Finance',
  'answer': 'AFBF-1: 3MK /COD +1.50%\nAFBF-2: 12 MK /COD +2.0%\nAFBF-3: As per SOC\nAFBF-4: As per SOC'},
 {'question': 'What is the markup repayment frequency in Ask Fauri Business Finance? for NUST Fauri Business Finance',
  'answer': 'AFBF-1: Monthly / Quarterly\nAFBF-2: Monthly / Quarterly\nAFBF-3: Upfront\nAFBF-4: Upfront'},
 {'question': 'What is the maximum tenure of Ask Fauri Business Finance? for NUST Fauri Business Finance',
  'answer': 'AFBF-1: Up to 1 Year\nAFBF-2: Up to 3 Year\nAFBF-3: Up to 30 Months\nAFBF-4: At Sight / Usance period 120 days'}]